# Google Scraping
---

In [191]:
import pandas as pd
import requests
import sys
sys.path.append('../')

from API_keys.config_omdb import omdb_key

import json
from pprint import pprint

from splinter import Browser
from bs4 import BeautifulSoup

import re
import time

In [192]:
from splinter import Browser
from bs4 import BeautifulSoup

### Scraping IMDB for Top 250 movies
___

In [196]:
!pip install requests beautifulsoup4


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [197]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [199]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize Chrome WebDriver
driver = webdriver.Chrome()

# Navigate to IMDb Top 250
url = "https://www.imdb.com/chart/top/"
driver.get(url)

# Scroll logic
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

soup = BeautifulSoup(driver.page_source, 'html.parser')
movie_data = []

# Find all movie entries
movies = soup.find_all('li', class_='ipc-metadata-list-summary-item')

for movie in movies:
    movie_dict = {}
    
    # Extract title and rank
    title = movie.find('h3', class_='ipc-title__text')
    if title:
        full_title = title.text.strip()
        # Extract rank from title (e.g., "1. The Shawshank Redemption")
        rank_match = re.match(r'^(\d+)\.', full_title)
        if rank_match:
            movie_dict['Rank'] = int(rank_match.group(1))
            # Remove rank from title
            title_text = re.sub(r'^\d+\.\s*', '', full_title)
            movie_dict['Title'] = title_text
        else:
            movie_dict['Rank'] = None
            movie_dict['Title'] = full_title
    else:
        movie_dict['Rank'] = None
        movie_dict['Title'] = 'Unknown'
    
    # Extract IMDB ID
    movie_link = movie.find('a', class_='ipc-title-link-wrapper')
    if movie_link:
        imdb_id = re.search(r'/title/(tt\d+)/', movie_link['href'])
        movie_dict['IMDB_ID'] = imdb_id.group(1) if imdb_id else 'Unknown'
    
    # Extract rating
    rating = movie.find('span', class_='ipc-rating-star--imdb')
    if rating:
        movie_dict['Rating'] = rating['aria-label'].split()[-1]
    
    # Extract year
    year = movie.find('span', class_='cli-title-metadata-item')
    movie_dict['Year'] = year.text if year else 'Unknown'
    
    movie_data.append(movie_dict)

driver.quit()

# Create DataFrame
df = pd.DataFrame(movie_data)

# Reorder columns
df = df[['Rank', 'Title', 'IMDB_ID', 'Rating', 'Year']]

# Sort by rank
df = df.sort_values('Rank')

print("Successfully scraped", len(df), "movies")
df.head()

# Save to CSV
df.to_csv('imdb_top_250.csv', index=False)

Successfully scraped 250 movies


In [200]:
df.head()

,Rank,Title,IMDB_ID,Rating,Year
0,1,The Shawshank Redemption,tt0111161,9.3,1994
1,2,The Godfather,tt0068646,9.2,1972
2,3,The Dark Knight,tt0468569,9.0,2008
3,4,The Godfather Part II,tt0071562,9.0,1974
4,5,12 Angry Men,tt0050083,9.0,1957


### Building the URL to scrape Google
---

In [213]:
movies = df['Title']
movies = movies.str.replace(r"[,:'.]", '', regex=True)

In [214]:
base_url = 'https://www.google.com/search?&q='

movie_list = movies.str.lower().str.replace(' ', '+', n = -1, case=None, regex=True)

query_url=[]

for movie in movie_list:
    query_url.append(f'{base_url}{movie}+watch+movie')

In [215]:
google_query_url_df = pd.DataFrame({'IMDb Rank': df['Rank'],
                                    'Movie Title': df['Title'],
                                    'Google Query URL' : query_url
                                   })

google_query_url_df = google_query_url_df.set_index(['IMDb Rank'])

google_query_url_df.head(10)

,Movie Title,Google Query URL
IMDb Rank,,
1,The Shawshank Redemption,https://www.google.com/search?&q=the+shawshank...
2,The Godfather,https://www.google.com/search?&q=the+godfather...
3,The Dark Knight,https://www.google.com/search?&q=the+dark+knig...
4,The Godfather Part II,https://www.google.com/search?&q=the+godfather...
5,12 Angry Men,https://www.google.com/search?&q=12+angry+men+...
6,The Lord of the Rings: The Return of the King,https://www.google.com/search?&q=the+lord+of+t...
7,Schindler's List,https://www.google.com/search?&q=schindlers+li...
8,Pulp Fiction,https://www.google.com/search?&q=pulp+fiction+...
9,The Lord of the Rings: The Fellowship of the Ring,https://www.google.com/search?&q=the+lord+of+t...


In [204]:
google_query_url_df.to_csv('../Output/Google_Query_Url.csv')

#### Sample Google scraping
---

In [205]:
#sample

sample = 'Inception'

base_url = 'https://www.google.com/search?&q='

query_url = (f'{base_url}{sample}+watch+movie')

browser.visit(query_url)

time.sleep(5)

soup = BeautifulSoup(browser.html, 'lxml')

streaming = []
title = []
price = []

results1 = soup.find_all('div', class_ = 'i3LlFf')

for result in results1:
    streaming.append(result.text)
    title.append(sample.capitalize())
    
results2 = soup.find_all('div', class_ = 'V8xno')

for result in results2:
    price.append(result.text)

Sample_Streaming_df = pd.DataFrame({'Title': title,
                                    'Streaming On' : streaming,
                                    'Price' : price
                                   })

Sample_Streaming_df['Price'] = Sample_Streaming_df['Price'].str.replace("$","", case = True, regex=True)
Sample_Streaming_df['Price'] = Sample_Streaming_df['Price'].str.replace("From ","", case = True, regex=True)

Sample_Streaming_df


AttributeError: 'WebDriver' object has no attribute 'visit'

In [ ]:
Sample_Streaming_df.to_csv('../Output/Sample_Google_Scraping.csv')

## Scraping Google for few movies at a time
---

In [ ]:
Streaming = []
Title = []
Price = []

count = 0

movies = google_query_url_df['Movie Title']

query_urls = google_query_url_df['Google Query URL']

base_url = 'https://www.google.com/search?&q='

In [ ]:
for i in range(8):
    
    query_url = (f'{base_url}{movies[count]}+watch+movie')

    browser.visit(query_url)

    time.sleep(3)

    soup = BeautifulSoup(browser.html, 'lxml')

    results1 = soup.find_all('div', class_ = 'i3LlFf')

    for result in results1:
        try:
            Streaming.append(result.text)
            Title.append(movies[count].capitalize())
        except:
            Streaming.append('Nan')
            Title.append('Nan')

    results2 = soup.find_all('div', class_ = 'V8xno')

    for result in results2:
        try:
            Price.append(result.text)
        except:
            Price.append('Nan')
            
    count = count + 1

print(f'{Title}, {Streaming}, {Price}')


In [ ]:
Streaming_df = pd.DataFrame({'Title': Title,
                             'Streaming On' : Streaming,
                             'Price' : Price
                            })

Streaming_df['Price'] = Streaming_df['Price'].str.replace("$","", case = True, regex=True)
Streaming_df['Price'] = Streaming_df['Price'].str.replace("From ","", case = True, regex=True)
Streaming_df['Title'] = Streaming_df['Title'].str.title()


In [ ]:
Streaming_df.head(35)

In [ ]:
# Removing the extra rows

Streaming_df_1 = Streaming_df.iloc[24:]
Streaming_df_1 = Streaming_df_1.reset_index()
Streaming_df_1 = Streaming_df_1.drop(['index'],axis = 1)
Streaming_df_1

,Title,Streaming On,Price
0,The Shawshank Redemption,YouTube,3.99
1,The Shawshank Redemption,iTunes,3.99
2,The Shawshank Redemption,Google Play Movies & TV,3.99
3,The Shawshank Redemption,Vudu,3.99
4,The Shawshank Redemption,Amazon Prime Video,3.99
...,...,...,...
794,The Thing,iTunes,3.99
795,The Thing,Google Play Movies & TV,3.99
796,The Thing,Vudu,3.99
797,The Thing,Amazon Prime Video,3.99


In [ ]:
Streaming_df_1.to_csv('../Output/Google_Scraping_1.csv')

### Scraped 164 movies so far
---

### Scraping remaining movies

In [ ]:
Streaming = []
Title = []
Price = []

count = 164

movies = google_query_url_df['Movie Title']

query_urls = google_query_url_df['Google Query URL']

base_url = 'https://www.google.com/search?&q='

In [ ]:
for i in range(6):
    
    query_url = (f'{base_url}{movies[count]}+watch+movie')

    browser.visit(query_url)

    time.sleep(3)

    soup = BeautifulSoup(browser.html, 'lxml')

    results1 = soup.find_all('div', class_ = 'i3LlFf')

    for result in results1:
        try:
            Streaming.append(result.text)
            Title.append(movies[count].capitalize())
        except:
            Streaming.append('Nan')
            Title.append('Nan')

    results2 = soup.find_all('div', class_ = 'V8xno')

    for result in results2:
        try:
            Price.append(result.text)
        except:
            Price.append('Nan')
            
    count = count + 1

#print(f'{Title}, {Streaming}, {Price}')


In [ ]:
print(f'Scraped {count} movies so far')

Scraped 250 movies so far


In [ ]:
Streaming_df_2 = pd.DataFrame({'Title': Title,
                               'Streaming On' : Streaming,
                               'Price' : Price
                              })

Streaming_df_2['Price'] = Streaming_df_2['Price'].str.replace("$","", case = True, regex=True)
Streaming_df_2['Price'] = Streaming_df_2['Price'].str.replace("From ","", case = True, regex=True)
Streaming_df_2['Title'] = Streaming_df_2['Title'].str.title()

Streaming_df_2

,Title,Streaming On,Price
0,Gone With The Wind,YouTube,3.99
1,Gone With The Wind,Vudu,3.99
2,Gone With The Wind,Google Play Movies & TV,3.99
3,Gone With The Wind,iTunes,3.99
4,Gone With The Wind,Amazon Prime Video,3.99
...,...,...,...
372,Akira,Vudu,2.99
373,Akira,iTunes,5.99
374,Akira,Hulu,Subscription
375,Throne Of Blood,iTunes,3.99


In [ ]:
Streaming_df_2.to_csv('../Output/Google_Scraping_2.csv')

### Scraped Google for 250 movies
---

## Concatenating the two dataframes 
---

In [ ]:
Complete_Stremaing_df = pd.concat([Streaming_df_1, Streaming_df_2], ignore_index=True)
Complete_Stremaing_df

,Title,Streaming On,Price
0,The Shawshank Redemption,YouTube,3.99
1,The Shawshank Redemption,iTunes,3.99
2,The Shawshank Redemption,Google Play Movies & TV,3.99
3,The Shawshank Redemption,Vudu,3.99
4,The Shawshank Redemption,Amazon Prime Video,3.99
...,...,...,...
1171,Akira,Vudu,2.99
1172,Akira,iTunes,5.99
1173,Akira,Hulu,Subscription
1174,Throne Of Blood,iTunes,3.99


In [ ]:
Complete_Stremaing_df.to_csv('../Output/Complete_Google_Scraping.csv')

In [ ]:
import os
os.getcwd()

'/Users/swarnaguntaka/Desktop/ETL-Project'